<a href="https://colab.research.google.com/drive/19rcq6c3XCbICuvWv18LypqG-ll3yAA7g?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

##**ALGORITHM FOR MASSIVE DATASETS:**
##**MARKET BASKET ANALYSIS** 

---


#####Federico Fiorio
#####Marco Cavallari
#####2021/2022 

## Spark setting

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init("spark-3.0.3-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

## Kaggle APIs

________________________________________________________________________

In [ ]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"federicofiorio","key":"3c9fda0dd26cc20d5f652a577743142c"}'}

In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!ls ~/.kaggle

kaggle.json


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 8.28G/8.29G [01:00<00:00, 208MB/s]
100% 8.29G/8.29G [01:00<00:00, 147MB/s]


In [ ]:
!ls

kaggle.json
sample_data
spark-3.0.3-bin-hadoop2.7
spark-3.0.3-bin-hadoop2.7.tgz
ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip


In [ ]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0401_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0402_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0403_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0404_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0405_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0406_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0407_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0408_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0409_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0410_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0411_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0412_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0413_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0414_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0415_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0416_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

## Use the data

________________________________________________________________________

In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import warnings

In [ ]:
filename = r"0401_UkraineCombinedTweetsDeduped.csv.gzip"
df = pd.read_csv(filename, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df[0:2].T

,0,1
userid,16882774,3205296069
username,Yaniela,gregffff
acctdesc,"Animal lover, supports those who fight injusti...",NaN
location,Hawaii,NaN
following,1158,122
followers,392,881
totaltweets,88366,99853
usercreatedts,2008-10-21 07:34:04.000000,2015-04-25 11:24:34.000000
tweetid,1509681950042198030,1509681950151348229
tweetcreatedts,2022-04-01 00:00:00.000000,2022-04-01 00:00:00.000000


In [ ]:
df = df[['tweetid', 'text', 'hashtags','language']]
df.head()

,tweetid,text,hashtags,language
0,1509681950042198030,⚡The Ukrainian Air Force would like to address...,[],en
1,1509681950151348229,Chernihiv oblast. Ukrainians welcome their lib...,"[{'text': 'russianinvasion', 'indices': [77, 9...",en
2,1509681950683926556,America 🇺🇸 is preparing for something worse th...,"[{'text': 'RussianUkrainianWar', 'indices': [7...",en
3,1509681951116046336,JUST IN: #Anonymous has hacked &amp; released ...,"[{'text': 'Anonymous', 'indices': [25, 35]}]",en
4,1509681951304990720,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[],en


In [ ]:
pd.options.display.max_colwidth = 1200
print(df[df['tweetid']==	1509681950042198030]["text"],df[df['tweetid']==	1509681950042198030]["hashtags"])

0    ⚡The Ukrainian Air Force would like to address misinformation published in multiple Western media outlets regarding the situation in the 🇺🇦 sky and support from our @NATO allies. \nMore in 🧵(1/16)\n#ProtectUАSky #StopRussia #UkraineUnderAttaсk
Name: text, dtype: object 0    []
Name: hashtags, dtype: object


As we can see in the previous cell, the tweet has the hashtag #StopRussia #UkraineUnderAttaсk BUT the dataframe (1st tuple) sais there are no hashtag in the tweet. We will recover form this situation by extractin the hashatgs directly from the filed "text" and not considering the (wrong) attribute "hashtags".

In [ ]:
from pyspark.sql.types import *

df = df[['tweetid', 'text','language']] #Taking out the field hashtags

schema = StructType([StructField("tweetid", StringType(), True)\
                   ,StructField("text", StringType(), True)\
                   ,StructField("language", StringType(), True)])  #We will use this field to filter out non english tweets

#create spark dataframe using schema
df_spark = spark.createDataFrame(df,schema=schema)

In [ ]:
df_spark.show()

+-------------------+--------------------+--------+
|            tweetid|                text|language|
+-------------------+--------------------+--------+
|1509681950042198030|⚡The Ukrainian Ai...|      en|
|1509681950151348229|Chernihiv oblast....|      en|
|1509681950683926556|America 🇺🇸 is p...|      en|
|1509681951116046336|JUST IN: #Anonymo...|      en|
|1509681951304990720|***PUBLIC MINT NO...|      en|
|1509681952000937999|The Amazing story...|      en|
|1509681952978210849|&amp;quot;How we ...|      en|
|1509681953053843466|India's purchase ...|      en|
|1509681953091457035|The most basic te...|      en|
|1509681953418711050|"The image that R...|      en|
|1509681953418752008|#Russia’s Preside...|      en|
|1509681953636700160|Sad to read of th...|      en|
|1509681953947074560|الرئيس الروسي يصع...|      ar|
|1509681954488303623|Tank T-34 at the ...|      en|
|1509681954647543824|The #Anonymous co...|      en|
|1509681955519938561|#Internacional 🇷...|      es|
|15096819556038

We will now use a variable "rdd_text" to extract hashtags from the tweets texts

In [ ]:
import nltk
from nltk import word_tokenize
import re
nltk.download('punkt')

rdd = df_spark.rdd #to get the rdd from dataframe

rdd.take(10)
rdd=rdd.filter(lambda x: x[2] == "en") #filter out non english tweets


rdd_text = rdd.map(lambda x : (x[0], x[1])).map(lambda x:(x[0], word_tokenize(x[1]))) #id, word_tokenized
rdd_text.first()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


('1509681950042198030',
 ['⚡The',
  'Ukrainian',
  'Air',
  'Force',
  'would',
  'like',
  'to',
  'address',
  'misinformation',
  'published',
  'in',
  'multiple',
  'Western',
  'media',
  'outlets',
  'regarding',
  'the',
  'situation',
  'in',
  'the',
  '🇺🇦',
  'sky',
  'and',
  'support',
  'from',
  'our',
  '@',
  'NATO',
  'allies',
  '.',
  'More',
  'in',
  '🧵',
  '(',
  '1/16',
  ')',
  '#',
  'ProtectUАSky',
  '#',
  'StopRussia',
  '#',
  'UkraineUnderAttaсk'])

rdd_text.first() contains a key value pair where the key is the tweetid and the value is the text of the tweet tokenized. Now from rdd_text I wil extract only the hashtags.

In [ ]:
def processHashtags(tweet):
  hashtags = []
  for i,word in enumerate(tweet):
    if word == '#':
      try:
        hashtags.append(re.sub(r'[^\w\s]','',tweet[i+1]).lower()) #remove punctuation with regexp and put them lowercase
      except:
        pass
  return hashtags

hashtags_per_tweet = rdd_text.map(lambda x: (x[0], processHashtags(x[1])))
hashtags_per_tweet.first()

('1509681950042198030', ['protectuаsky', 'stoprussia', 'ukraineunderattaсk'])

hashtags_per_tweet.first() contains all the hashtags of the first tweet. As We saw earlier, The field "hashtags" of the first tweet in the original dataset was a empty list, which is wrong.

In [ ]:
#reconvert the rdd to spark dataframe (just fro visulaziation)
deptColumns = ["tweet","hashtag"]
new_df_spark = hashtags_per_tweet.toDF(deptColumns)
new_df_spark.printSchema()
new_df_spark.show(truncate=False)

root
 |-- tweet: string (nullable = true)
 |-- hashtag: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet              |hashtag                                                                                                                                                                   |
+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1509681950042198030|[protectuаsky, stoprussia, ukraineunderattaсk]                                                                                                                            |
|1509681950151348229|[russianinvasion, standwithukraine, ukraineunderattack, ukrainewillwin, putini

In [ ]:
hashtag=new_df_spark.select('hashtag')

In [ ]:
basket_file = hashtag.rdd.flatMap(list)

For sake of simplicity, We used a really small part of our dataset: just 500 baskets. Having said this, the following code should be able to scale up with the size of the dataset but given the already mentioned HW limitation, We decided to just use a "toy dataset".

In [ ]:
num_baskets = 500
basket_file = basket_file.take(num_baskets) 

In [ ]:
basket_file = spark.sparkContext.parallelize(basket_file)

In [ ]:
type(basket_file)

pyspark.rdd.RDD

In [ ]:
basket_file.take(10) #first 10 baskets/tweets

[['protectuаsky', 'stoprussia', 'ukraineunderattaсk'],
 ['russianinvasion',
  'standwithukraine',
  'ukraineunderattack',
  'ukrainewillwin',
  'putinisawarcriminal',
  'stopputin',
  'russianukrainianwar',
  'russiagohome',
  'россиясмотри',
  'нетвойне'],
 ['russianukrainianwar', 'china', 'taiwan'],
 ['anonymous', 'oprussia', 'ddosecrets'],
 ['nft', 'mint'],
 ['russia',
  'ukraine',
  'motivation',
  'netde',
  'edude',
  'delaware',
  'government',
  'usa'],
 ['ukraine', 'ukrainewar', 'russia', 'ukraineinvasion'],
 ['russian', 'moscow'],
 ['ukraine'],
 ['putin', 'medvedev', 'russia', 'ukraine']]

In [ ]:
threshold = 10  #threshold fixed empirically
#print(threshold,basket_file.count())

###APRIORI ALGORITHM

_______________________________________________________________________________

## Start apriori: PHASE 1

In [ ]:
#phase 1 calculate the freq of each item in the basket file
singleton=basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

##BETWEEN THE TWO PHASES

In [ ]:
#between the two phases: filter out all the non frequent singleton
freq_singleton=singleton.filter(lambda x: x[1]>=threshold)

## PHASE 2

In [ ]:
#phase 2.1: now create all the possible pairs made up by frequen singletons

from itertools import combinations
pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #without toLocalIterator error (reference https://stackoverflow.com/questions/32771737/convert-an-rdd-to-iterable-pyspark)

pairs[:100]

[('ukraineunderattaсk', 'russianinvasion'),
 ('ukraineunderattaсk', 'standwithukraine'),
 ('ukraineunderattaсk', 'ukrainewillwin'),
 ('ukraineunderattaсk', 'stopputin'),
 ('ukraineunderattaсk', 'russianukrainianwar'),
 ('ukraineunderattaсk', 'russiagohome'),
 ('ukraineunderattaсk', 'нетвойне'),
 ('ukraineunderattaсk', 'anonymous'),
 ('ukraineunderattaсk', 'putin'),
 ('ukraineunderattaсk', 'ukrainian'),
 ('ukraineunderattaсk', 'russiaukrainewar'),
 ('ukraineunderattaсk', 'mariupol'),
 ('ukraineunderattaсk', 'ukrainerussianwar'),
 ('ukraineunderattaсk', 'ukrainerussiawar'),
 ('ukraineunderattaсk', 'settodraw'),
 ('ukraineunderattaсk', 'tanzaniaproject'),
 ('ukraineunderattaсk', 'africa'),
 ('ukraineunderattaсk', 'mining'),
 ('ukraineunderattaсk', 'kherson'),
 ('ukraineunderattaсk', 'protectuаsky'),
 ('ukraineunderattaсk', 'stoprussia'),
 ('ukraineunderattaсk', 'ukraineunderattack'),
 ('ukraineunderattaсk', 'putinisawarcriminal'),
 ('ukraineunderattaсk', 'россиясмотри'),
 ('ukraineunderat

Now for each basket, if one of the prevously generated pairs (made of freq singletons) is inside a basket, We generate a tuple (pair,1). Then We flat the results.

In [ ]:
flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in pairs if set(pair).issubset(set(x))]).flatMap(lambda x: x).cache()

In [ ]:
reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)

In [ ]:
freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
result_APRIORI = list(freq_pairs.map(lambda x: x).toLocalIterator())

In [ ]:
import time
startTime = time.time()
print(time.time()-startTime)
freq_pairs.collect()

8.392333984375e-05


[(('protectuаsky', 'stoprussia'), 20),
 (('russianinvasion', 'standwithukraine'), 20),
 (('russianinvasion', 'ukrainewillwin'), 20),
 (('russianinvasion', 'stopputin'), 20),
 (('russianinvasion', 'russianukrainianwar'), 20),
 (('russianinvasion', 'russiagohome'), 20),
 (('russianinvasion', 'нетвойне'), 20),
 (('standwithukraine', 'ukrainewillwin'), 20),
 (('standwithukraine', 'stopputin'), 23),
 (('standwithukraine', 'russianukrainianwar'), 20),
 (('standwithukraine', 'russiagohome'), 20),
 (('standwithukraine', 'нетвойне'), 20),
 (('ukrainewillwin', 'stopputin'), 20),
 (('ukrainewillwin', 'russianukrainianwar'), 20),
 (('ukrainewillwin', 'russiagohome'), 20),
 (('ukrainewillwin', 'нетвойне'), 20),
 (('stopputin', 'russianukrainianwar'), 20),
 (('stopputin', 'russiagohome'), 20),
 (('stopputin', 'нетвойне'), 20),
 (('russianukrainianwar', 'russiagohome'), 20),
 (('russianukrainianwar', 'нетвойне'), 20),
 (('russiagohome', 'нетвойне'), 20),
 (('ukraineunderattack', 'putinisawarcriminal'

## PCY ALGORITHM 

#First Pass

In [ ]:
singleton = basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

In [ ]:
from sys import hash_info

HASH_TABLE_SIZE = 50000 
hash_table = np.zeros(HASH_TABLE_SIZE, dtype=int)

def hashing(pair_to_hash):
  return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZE)

In [ ]:
hashtable_rdd = basket_file.map(lambda basket: [(hashing(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                           .reduceByKey(lambda a,b: a+b)

In the previous line of code, for each basket, all the pairs in it will be hashed. In hashtable_rdd  will be stored a list of key value pairs where the key is the bucket in which a pair was hashed and the value is the number of pairs hashed in that bucket.

#SECOND PASS

In [ ]:
hashtable_list = list(hashtable_rdd.map(lambda x: x).toLocalIterator())

for pair in hashtable_list:
  hash_table[pair[0]] = pair[1]

bitmap_freq = [hash_table[i]>=threshold for i in range(HASH_TABLE_SIZE)] #bitmap_freq[i]==False if hash_table[i]<threshold, True otherwise


In [ ]:
from itertools import combinations

freq_singleton = singleton.filter(lambda x: x[1]>=threshold)  
freq_singleton_list = list(freq_singleton.map(lambda x: x).toLocalIterator())                         
pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #formed from freq singletons

In [ ]:
#CONVERT PAIRS INTO RDD
pairs_rdd = spark.sparkContext.parallelize(pairs)

In [ ]:
#candidate pairs = pairs of freq singleton and the pair is freq in the hash_table
candidate_pairs = pairs_rdd.filter(lambda x : bitmap_freq[hashing(x)] == True)
candidate_pairs_list = list(candidate_pairs.map(lambda x: x).toLocalIterator()) 
candidate_pairs_list

[('ukraineunderattaсk', 'protectuаsky'),
 ('ukraineunderattaсk', 'stoprussia'),
 ('russianinvasion', 'standwithukraine'),
 ('russianinvasion', 'ukrainewillwin'),
 ('russianinvasion', 'stopputin'),
 ('russianinvasion', 'russianukrainianwar'),
 ('russianinvasion', 'russiagohome'),
 ('russianinvasion', 'нетвойне'),
 ('russianinvasion', 'ukrainerussiawar'),
 ('russianinvasion', 'ukraineunderattack'),
 ('russianinvasion', 'putinisawarcriminal'),
 ('russianinvasion', 'россиясмотри'),
 ('standwithukraine', 'ukrainewillwin'),
 ('standwithukraine', 'stopputin'),
 ('standwithukraine', 'russianukrainianwar'),
 ('standwithukraine', 'russiagohome'),
 ('standwithukraine', 'нетвойне'),
 ('standwithukraine', 'ukraineunderattack'),
 ('standwithukraine', 'putinisawarcriminal'),
 ('standwithukraine', 'россиясмотри'),
 ('ukrainewillwin', 'stopputin'),
 ('ukrainewillwin', 'russianukrainianwar'),
 ('ukrainewillwin', 'russiagohome'),
 ('ukrainewillwin', 'нетвойне'),
 ('ukrainewillwin', 'ukraineunderattack'),

In [ ]:
#check against threshold
flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in candidate_pairs_list if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()

reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
result_PCY = list(freq_pairs.map(lambda x: x).toLocalIterator())
freq_pairs.collect()

[(('protectuаsky', 'stoprussia'), 20),
 (('russianinvasion', 'standwithukraine'), 20),
 (('russianinvasion', 'ukrainewillwin'), 20),
 (('russianinvasion', 'stopputin'), 20),
 (('russianinvasion', 'russianukrainianwar'), 20),
 (('russianinvasion', 'russiagohome'), 20),
 (('russianinvasion', 'нетвойне'), 20),
 (('standwithukraine', 'ukrainewillwin'), 20),
 (('standwithukraine', 'stopputin'), 23),
 (('standwithukraine', 'russianukrainianwar'), 20),
 (('standwithukraine', 'russiagohome'), 20),
 (('standwithukraine', 'нетвойне'), 20),
 (('ukrainewillwin', 'stopputin'), 20),
 (('ukrainewillwin', 'russianukrainianwar'), 20),
 (('ukrainewillwin', 'russiagohome'), 20),
 (('ukrainewillwin', 'нетвойне'), 20),
 (('stopputin', 'russianukrainianwar'), 20),
 (('stopputin', 'russiagohome'), 20),
 (('stopputin', 'нетвойне'), 20),
 (('russianukrainianwar', 'russiagohome'), 20),
 (('russianukrainianwar', 'нетвойне'), 20),
 (('russiagohome', 'нетвойне'), 20),
 (('ukraineunderattack', 'putinisawarcriminal'

In [ ]:
set(result_PCY) == set(result_APRIORI)

True

### MULTISTAGE

In [ ]:
HASH_TABLE_SIZE = 50000
hash_table_1 = np.zeros(HASH_TABLE_SIZE, dtype=int)
hash_table_2 = np.zeros(HASH_TABLE_SIZE, dtype=int)

def hashing_1(pair_to_hash):
  return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZE)


def hashing_2(pair_to_hash):
  return int((abs(hash(frozenset(pair_to_hash)))*3 /5 +9 *2 )% HASH_TABLE_SIZE)  #just create another hash function

hashing_1(('russianarmy', 'chernobyl')), hashing_2(('russianarmy', 'chernobyl'))

(27890, 6480)

In [ ]:
#1st pass multistage = PCY

singleton = basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

from itertools import combinations
hashtable_rdd = basket_file.map(lambda basket: [(hashing(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                           .reduceByKey(lambda a,b: a+b)

hashtable_list = list(hashtable_rdd.map(lambda x: x).toLocalIterator())

for pair in hashtable_list:
  hash_table_1[pair[0]] = pair[1]


### 2nd Stage

In [ ]:
from itertools import combinations
bitmap_1 = [hash_table_1[i]>=threshold for i in range(HASH_TABLE_SIZE)]

freq_singleton = singleton.filter(lambda x: x[1]>=threshold)                           
pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #All pairs formed by freqent singletons

I have bitmap_1, I have the freq singletons, and the pairs from freq singletons. Now I need to check against bitmap_1 the pairs formed from freq singlentons, than the pairs hashed in frequent bucket from bitmap_1 will be hashed again to create bitmap 2.

In [ ]:
#CONVERT PAIRS INTO RDD
pairs_rdd = spark.sparkContext.parallelize(pairs) #Formed from freq singletons

In [ ]:
#check against bitmap 1
pairs_first_check = pairs_rdd.filter(lambda x : bitmap_1[hashing_1(x)] == True) #pairs from freq singletons and resulted freq in bitmap_1
#hash them again to hashtable 2

pairs_first_check_list = list(pairs_first_check .map(lambda x: x).toLocalIterator())#creating all pairs

#2nd scan: I hash the pairs that are formed by frequent singleton and hased in a freqient bcket from bitmap_1
hashtable_rdd_2 = basket_file.map(lambda x: [(hashing_2(pair),1) for pair in pairs_first_check_list if set(pair).issubset(set(x))])\
                            .flatMap(lambda x: x).cache().reduceByKey(lambda a,b: a+b)

hashtable_list_2 = list(hashtable_rdd_2.map(lambda x: x).toLocalIterator()) 

for pair in hashtable_list_2:
  hash_table_2[pair[0]] = pair[1]


### 3rd Stage

In [ ]:
bitmap_2 = [hash_table_2[i]>=threshold for i in range(HASH_TABLE_SIZE)]

I have now bitmap_1 , bitmap_2 and all the pairs from freq singletons (pairs_rdd): if pair (made of freq singletons) is in bitmap_1 and bitmap_2 it means it's a CANDIDATE pair

In [ ]:
candidate_pairs_1 = pairs_rdd.filter(lambda x : bitmap_1[hashing_1(x)] == True)
candidate_pairs = candidate_pairs_1.filter(lambda x : bitmap_2[hashing_2(x)] == True) 
candidate_pairs_list = list(candidate_pairs.map(lambda x: x).toLocalIterator())

In [ ]:
flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in candidate_pairs_list if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()

reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
result_MULTI_STAGE=list(freq_pairs.map(lambda x: x).toLocalIterator())
freq_pairs.collect()

[(('protectuаsky', 'stoprussia'), 20),
 (('russianinvasion', 'standwithukraine'), 20),
 (('russianinvasion', 'ukrainewillwin'), 20),
 (('russianinvasion', 'stopputin'), 20),
 (('russianinvasion', 'russianukrainianwar'), 20),
 (('russianinvasion', 'russiagohome'), 20),
 (('russianinvasion', 'нетвойне'), 20),
 (('standwithukraine', 'ukrainewillwin'), 20),
 (('standwithukraine', 'stopputin'), 23),
 (('standwithukraine', 'russianukrainianwar'), 20),
 (('standwithukraine', 'russiagohome'), 20),
 (('standwithukraine', 'нетвойне'), 20),
 (('ukrainewillwin', 'stopputin'), 20),
 (('ukrainewillwin', 'russianukrainianwar'), 20),
 (('ukrainewillwin', 'russiagohome'), 20),
 (('ukrainewillwin', 'нетвойне'), 20),
 (('stopputin', 'russianukrainianwar'), 20),
 (('stopputin', 'russiagohome'), 20),
 (('stopputin', 'нетвойне'), 20),
 (('russianukrainianwar', 'russiagohome'), 20),
 (('russianukrainianwar', 'нетвойне'), 20),
 (('russiagohome', 'нетвойне'), 20),
 (('ukraineunderattack', 'putinisawarcriminal'

In [ ]:
set(result_MULTI_STAGE) == set(result_PCY)

True

## MULTI-HASH

1ST STAGE

In [ ]:
singleton = basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

In [ ]:
HASH_TABLE_SIZE = 25000
hash_table_1 = np.zeros(HASH_TABLE_SIZE, dtype=int)
hash_table_2 = np.zeros(HASH_TABLE_SIZE, dtype=int)

def hashing_1(pair_to_hash):
  return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZE)


def hashing_2(pair_to_hash):
  return int((abs(hash(frozenset(pair_to_hash)))*3 /5 +9 *2 )% HASH_TABLE_SIZE)  #just create another hash function

hashing_1(('russianarmy', 'chernobyl')), hashing_2(('russianarmy', 'chernobyl'))

(2890, 6480)

In [ ]:
#hashing all the pairs of the basket fle with the two hashing functions

hashtable_rdd_1 = basket_file.map(lambda basket: [(hashing_1(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                           .reduceByKey(lambda a,b: a+b)

hashtable_rdd_2 = basket_file.map(lambda basket: [(hashing_2(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                           .reduceByKey(lambda a,b: a+b)



#2ND PASS

In [ ]:
hashtable_list_1 = list(hashtable_rdd_1.map(lambda x: x).toLocalIterator())

for pair in hashtable_list_1:
  hash_table_1[pair[0]] = pair[1]

bitmap_freq_1 = [hash_table_1[i]>=threshold for i in range(HASH_TABLE_SIZE)] #bitmap_freq_1 from hash_table_1 (obtained from hashtable_rdd_1)


hashtable_list_2 = list(hashtable_rdd_2.map(lambda x: x).toLocalIterator())

for pair in hashtable_list_2:
  hash_table_2[pair[0]] = pair[1]

bitmap_freq_2 = [hash_table_2[i]>=threshold for i in range(HASH_TABLE_SIZE)] #bitmap_freq_2 from hash_table_2 (obtained from hashtable_rdd_2)



In [ ]:
freq_singleton = singleton.filter(lambda x: x[1]>=threshold) 
                          
pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #pairs of freq singletons

In [ ]:
#CONVERT PAIRS INTO RDD
pairs_rdd = spark.sparkContext.parallelize(pairs)

In [ ]:
#candidate pairs = pairs of freq singleton and the pair is freq in both the hash_tables

candidate_pairs = pairs_rdd.filter(lambda x : bitmap_freq_1[hashing_1(x)] == True)
candidate_pairs_2 = candidate_pairs.filter(lambda x : bitmap_freq_2[hashing_2(x)] == True)

candidate_pairs_list = list(candidate_pairs_2.map(lambda x: x).toLocalIterator()) 
candidate_pairs_list[:10]

[('ukraineunderattaсk', 'protectuаsky'),
 ('ukraineunderattaсk', 'stoprussia'),
 ('russianinvasion', 'standwithukraine'),
 ('russianinvasion', 'ukrainewillwin'),
 ('russianinvasion', 'stopputin'),
 ('russianinvasion', 'russianukrainianwar'),
 ('russianinvasion', 'russiagohome'),
 ('russianinvasion', 'нетвойне'),
 ('russianinvasion', 'ukraineunderattack'),
 ('russianinvasion', 'putinisawarcriminal')]

In [ ]:
flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in candidate_pairs_list if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()

reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
result_MULTI_HASH = list(freq_pairs.map(lambda x: x).toLocalIterator())
result_MULTI_HASH_sorted = list(freq_pairs.sortBy(lambda x: x[1]).toLocalIterator())

freq_pairs.collect()

[(('protectuаsky', 'stoprussia'), 20),
 (('russianinvasion', 'standwithukraine'), 20),
 (('russianinvasion', 'ukrainewillwin'), 20),
 (('russianinvasion', 'stopputin'), 20),
 (('russianinvasion', 'russianukrainianwar'), 20),
 (('russianinvasion', 'russiagohome'), 20),
 (('russianinvasion', 'нетвойне'), 20),
 (('standwithukraine', 'ukrainewillwin'), 20),
 (('standwithukraine', 'stopputin'), 23),
 (('standwithukraine', 'russianukrainianwar'), 20),
 (('standwithukraine', 'russiagohome'), 20),
 (('standwithukraine', 'нетвойне'), 20),
 (('ukrainewillwin', 'stopputin'), 20),
 (('ukrainewillwin', 'russianukrainianwar'), 20),
 (('ukrainewillwin', 'russiagohome'), 20),
 (('ukrainewillwin', 'нетвойне'), 20),
 (('stopputin', 'russianukrainianwar'), 20),
 (('stopputin', 'russiagohome'), 20),
 (('stopputin', 'нетвойне'), 20),
 (('russianukrainianwar', 'russiagohome'), 20),
 (('russianukrainianwar', 'нетвойне'), 20),
 (('russiagohome', 'нетвойне'), 20),
 (('ukraineunderattack', 'putinisawarcriminal'

In [ ]:
set(result_MULTI_HASH) == set(result_MULTI_STAGE)

True

## SON

In [ ]:
# define Apriori function (same lines of code previously seen) to use in SON
from itertools import combinations
def Apriori(basket_file, threshold):
  singleton=basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b) #1st phase
  freq_singleton=singleton.filter(lambda x: x[1]>=threshold)#between 1st and 2nd phase
  pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #start of the 2nd phase
  flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in pairs if set(pair).issubset(set(x))]).flatMap(lambda x: x).cache()
  reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
  freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
  return freq_pairs

In [ ]:
#We need to define the size of the chunk for dividing the basket file
NUMBER_OF_CHUNKS = 2 
num_baskets = basket_file.count()

chunks = basket_file.repartition(NUMBER_OF_CHUNKS).glom().collect() 

SIZE_CHUNK = num_baskets/NUMBER_OF_CHUNKS
print(SIZE_CHUNK)

250.0


In [ ]:
all_pairs = spark.sparkContext.parallelize([]) #empty RDD
scaled_threshold = 0.9 * (SIZE_CHUNK/num_baskets) * threshold

for chunk in chunks:
  rdd_chunk = spark.sparkContext.parallelize(chunk) #rdd_chun contain thecurrent chunk
  freq_pairs = Apriori(rdd_chunk, threshold=scaled_threshold)
  all_pairs = all_pairs.union(freq_pairs) 
  
#all pairs at the end of the for will contain the results of Apriori run on the two chunks

all_pairs = all_pairs.map(lambda x : x[0]).distinct()  #elimination of duplicates (from (a,b),(a,b) to (a,b))
candidate_pairs_list = list(all_pairs.map(lambda x: x).toLocalIterator()) #convert to list


# It is possible that Apriori output from a chunk a frequent pair ("Hello","Computer) and Apriori output ("Hello","Computer) for another chunk.
#We need to delete also the duplicates of this from.

candidate_pairs = candidate_pairs_list

for i, pair_1 in enumerate(candidate_pairs_list):
  for j, pair_2 in enumerate(candidate_pairs_list):
    if (i!= j and frozenset(pair_1) == frozenset(pair_2)): # I got something like (a,b) (b,a) but I'm intrested only in one of them.
      candidate_pairs.remove(pair_2) #remove pair_2 from list


## 2nd map reduce

In [ ]:
#CHECK AGAINST FP

final_rdd = spark.sparkContext.parallelize([]) #empty RDD

for chunk in chunks:
  rdd_chunk = spark.sparkContext.parallelize(chunk)
  flatted_couples_chunk = rdd_chunk.map(lambda x: [(pair,1) for pair in candidate_pairs if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()
  final_rdd = final_rdd.union(flatted_couples_chunk)


reduced_elements = final_rdd.reduceByKey(lambda a, b: a + b)
freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache() 
result_SON = list(freq_pairs.map(lambda x: x).toLocalIterator())
freq_pairs.collect()

[(('protectuаsky', 'stoprussia'), 20),
 (('russianinvasion', 'standwithukraine'), 20),
 (('ukrainewillwin', 'stopputin'), 20),
 (('ukrainewillwin', 'russiagohome'), 20),
 (('ukrainewillwin', 'нетвойне'), 20),
 (('russiagohome', 'нетвойне'), 20),
 (('ukraineunderattack', 'putinisawarcriminal'), 20),
 (('ukraineunderattack', 'россиясмотри'), 20),
 (('stopputin', 'standwithukraine'), 23),
 (('russianukrainianwar', 'standwithukraine'), 20),
 (('russianukrainianwar', 'russianinvasion'), 20),
 (('russianukrainianwar', 'ukrainewillwin'), 20),
 (('nft', 'mint'), 33),
 (('russia', 'ukraine'), 29),
 (('ukraine', 'kyiv'), 22),
 (('settodraw', 'tanzaniaproject'), 10),
 (('settodraw', 'africa'), 10),
 (('tanzaniaproject', 'mining'), 10),
 (('russianinvasion', 'ukraineunderattack'), 20),
 (('russianinvasion', 'putinisawarcriminal'), 20),
 (('russianinvasion', 'россиясмотри'), 20),
 (('standwithukraine', 'ukraineunderattack'), 20),
 (('russianukrainianwar', 'ukraineunderattack'), 20),
 (('russianukra

In [ ]:
set(result_SON) == set(result_MULTI_HASH)

False

# TESTING ON THE WHOLE DATASET

In [ ]:
hashtag=new_df_spark.select('hashtag')
threshold = 7000  #threshold fixed empirically
basket_file = hashtag.rdd.flatMap(list)
basket_file.count()

254626

In [ ]:
import time
def analysis(algorithm,basket_file):
  num_pairs=[]
  times=[]
  pairs=[]

  for t in [1000,3000,5000,7000]:
    startTime = time.time()
    if algorithm =="Apriori":
      freq_pairs = Apriori(basket_file, threshold=t)
    elif algorithm =="PCY":
      freq_pairs = PCY(basket_file, threshold=t, HASH_TABLE_SIZE=50000) #HASHTABLE SIZE
    elif algorithm =="Multistage":
      freq_pairs = Multistage(basket_file, threshold=t, HASH_TABLE_SIZE=25000)  #HASHTABLE SIZE
    elif algorithm =="MULTI_HASH":
      freq_pairs = MULTI_HASH(basket_file, threshold=t, HASH_TABLE_SIZEs=25000) #HASHTABLE SIZE
    elif algorithm =="SON":
      freq_pairs = SON(basket_file, threshold=t)    #NUMBER OF CHUNKS
    f=freq_pairs.collect()
    c=freq_pairs.count()
    t=time.time()-startTime

    num_pairs.append(c)
    pairs.append(f)
    times.append(t)

    print("*")
  return num_pairs,pairs,times

## RE-WRITE THE ALGORITHMS AS FUNCTION (ALREADY DONE FOR APRIORI)

In [ ]:
from sys import hash_info
from itertools import combinations

def hashing(pair_to_hash):
  return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZE)

def PCY(basket_file, threshold, HASH_TABLE_SIZE):
  def hashing(pair_to_hash):
    return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZE)
  #1st pass:
  singleton = basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)
  HASH_TABLE_SIZE = 50000 
  hash_table = np.zeros(HASH_TABLE_SIZE, dtype=int)

  hashtable_rdd = basket_file.map(lambda basket: [(hashing(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                           .reduceByKey(lambda a,b: a+b)

  #2st pass:
  hashtable_list = list(hashtable_rdd.map(lambda x: x).toLocalIterator())

  for pair in hashtable_list:
    hash_table[pair[0]] = pair[1]

  bitmap_freq = [hash_table[i]>=threshold for i in range(HASH_TABLE_SIZE)]

  freq_singleton = singleton.filter(lambda x: x[1]>=threshold)  
  freq_singleton_list = list(freq_singleton.map(lambda x: x).toLocalIterator())                         
  pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #formed from freq singletons
  #CONVERT PAIRS INTO RDD
  pairs_rdd = spark.sparkContext.parallelize(pairs)
  candidate_pairs = pairs_rdd.filter(lambda x : bitmap_freq[hashing(x)] == True)
  candidate_pairs_list = list(candidate_pairs.map(lambda x: x).toLocalIterator()) 

  #check against threshold
  flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in candidate_pairs_list if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()

  reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
  freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
  return freq_pairs


In [ ]:
from itertools import combinations



def Multistage(basket_file, threshold, HASH_TABLE_SIZE):
  hash_table_1 = np.zeros(HASH_TABLE_SIZE, dtype=int)
  hash_table_2 = np.zeros(HASH_TABLE_SIZE, dtype=int)

  def hashing_1(pair_to_hash):
    return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZE)

  def hashing_2(pair_to_hash):
    return int((abs(hash(frozenset(pair_to_hash)))*3 /5 +9 *2 )% HASH_TABLE_SIZE)  #just create another hash function
  
  #1st stage
  singleton = basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)

  
  hashtable_rdd = basket_file.map(lambda basket: [(hashing_1(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                           .reduceByKey(lambda a,b: a+b)

  hashtable_list = list(hashtable_rdd.map(lambda x: x).toLocalIterator())

  for pair in hashtable_list:
    hash_table_1[pair[0]] = pair[1]

  #2nd stage
  bitmap_1 = [hash_table_1[i]>=threshold for i in range(HASH_TABLE_SIZE)]

  freq_singleton = singleton.filter(lambda x: x[1]>=threshold)                           
  pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #All pairs formed by freqent singletons

  #CONVERT PAIRS INTO RDD
  pairs_rdd = spark.sparkContext.parallelize(pairs) #Formed from freq singletons

  #check against bitmap 1
  pairs_first_check = pairs_rdd.filter(lambda x : bitmap_1[hashing_1(x)] == True) #pairs from freq singletons and resulted freq in bitmap_1
  #hash them again to hashtable 2

  pairs_first_check_list = list(pairs_first_check .map(lambda x: x).toLocalIterator())#creating all pairs

  #2nd scan: I hash the pairs that are formed by frequent singleton and hased in a freqient bcket from bitmap_1
  hashtable_rdd_2 = basket_file.map(lambda x: [(hashing_2(pair),1) for pair in pairs_first_check_list if set(pair).issubset(set(x))])\
                              .flatMap(lambda x: x).cache().reduceByKey(lambda a,b: a+b)

  hashtable_list_2 = list(hashtable_rdd_2.map(lambda x: x).toLocalIterator()) 

  for pair in hashtable_list_2:
    hash_table_2[pair[0]] = pair[1]
    
  #3rd stage
  bitmap_2 = [hash_table_2[i]>=threshold for i in range(HASH_TABLE_SIZE)]
  candidate_pairs_1 = pairs_rdd.filter(lambda x : bitmap_1[hashing_1(x)] == True)
  candidate_pairs = candidate_pairs_1.filter(lambda x : bitmap_2[hashing_2(x)] == True) 
  candidate_pairs_list = list(candidate_pairs.map(lambda x: x).toLocalIterator())

  flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in candidate_pairs_list if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()

  reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
  freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()
  return freq_pairs

#Multistage(basket_file, threshold, 25000).collect()

In [ ]:
def MULTI_HASH(basket_file, threshold, HASH_TABLE_SIZEs):
  singleton = basket_file.flatMap(list).map(lambda item: (item,1)).reduceByKey(lambda a,b: a+b)
  hash_table_1 = np.zeros(HASH_TABLE_SIZEs, dtype=int)
  hash_table_2 = np.zeros(HASH_TABLE_SIZEs, dtype=int)

  def hashing_1(pair_to_hash):
    return int(abs(hash(frozenset(pair_to_hash))) % HASH_TABLE_SIZEs)

  def hashing_2(pair_to_hash):
    return int((abs(hash(frozenset(pair_to_hash)))*3 /5 +9 *2 )% HASH_TABLE_SIZEs)  #just create another hash function

  hashtable_rdd_1 = basket_file.map(lambda basket: [(hashing_1(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                            .reduceByKey(lambda a,b: a+b)

  hashtable_rdd_2 = basket_file.map(lambda basket: [(hashing_2(pair), 1) for pair in list(combinations(basket, 2))]).flatMap(lambda x: x).cache()\
                            .reduceByKey(lambda a,b: a+b)

  hashtable_list_1 = list(hashtable_rdd_1.map(lambda x: x).toLocalIterator())

  for pair in hashtable_list_1:
    hash_table_1[pair[0]] = pair[1]

  bitmap_freq_1 = [hash_table_1[i]>=threshold for i in range(HASH_TABLE_SIZEs)] #bitmap_freq_1 from hash_table_1 (obtained from hashtable_rdd_1)


  hashtable_list_2 = list(hashtable_rdd_2.map(lambda x: x).toLocalIterator())

  for pair in hashtable_list_2:
    hash_table_2[pair[0]] = pair[1]

  bitmap_freq_2 = [hash_table_2[i]>=threshold for i in range(HASH_TABLE_SIZEs)] #bitmap_freq_2 from hash_table_2 (obtained from hashtable_rdd_2)

  freq_singleton = singleton.filter(lambda x: x[1]>=threshold) 
  pairs=list(combinations(freq_singleton.map(lambda x: x[0]).toLocalIterator(),2)) #pairs of freq singletons
  #CONVERT PAIRS INTO RDD
  pairs_rdd = spark.sparkContext.parallelize(pairs)
  #candidate pairs = pairs of freq singleton and the pair is freq in both the hash_tables

  candidate_pairs = pairs_rdd.filter(lambda x : bitmap_freq_1[hashing_1(x)] == True)
  candidate_pairs_2 = candidate_pairs.filter(lambda x : bitmap_freq_2[hashing_2(x)] == True)

  candidate_pairs_list = list(candidate_pairs_2.map(lambda x: x).toLocalIterator()) 
  flatted_couples = basket_file.map(lambda x: [(pair,1) for pair in candidate_pairs_list if set(pair).\
                                             issubset(set(x))]).flatMap(lambda x: x).cache()

  reduced_elements = flatted_couples.reduceByKey(lambda a, b: a + b)
  freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache()

  return freq_pairs

#MULTI_HASH(basket_file, threshold, 25000).collect()

In [ ]:
def SON(basket_file, threshold, NUMBER_OF_CHUNKS=2):
  num_baskets = basket_file.count()
  chunks = basket_file.repartition(NUMBER_OF_CHUNKS).glom().collect() 
  SIZE_CHUNK = num_baskets/NUMBER_OF_CHUNKS
  all_pairs = spark.sparkContext.parallelize([]) #empty RDD
  scaled_threshold = 0.9 * (SIZE_CHUNK/num_baskets) * threshold

  for chunk in chunks:
    rdd_chunk = spark.sparkContext.parallelize(chunk) #rdd_chun contain thecurrent chunk
    freq_pairs = Apriori(rdd_chunk, threshold=scaled_threshold)
    all_pairs = all_pairs.union(freq_pairs) 
    
  #all pairs at the end of the for will contain the results of Apriori run on the two chunks

  all_pairs = all_pairs.map(lambda x : x[0]).distinct()  #elimination of duplicates (from (a,b),(a,b) to (a,b))
  candidate_pairs_list = list(all_pairs.map(lambda x: x).toLocalIterator()) #convert to list


  # It is possible that Apriori output from a chunk a frequent pair ("Hello","Computer) and Apriori output ("Hello","Computer) for another chunk.
  #We need to delete also the duplicates of this from.

  candidate_pairs = candidate_pairs_list

  for i, pair_1 in enumerate(candidate_pairs_list):
    for j, pair_2 in enumerate(candidate_pairs_list):
      if (i!= j and frozenset(pair_1) == frozenset(pair_2)): # I got something like (a,b) (b,a) but I'm intrested only in one of them.
        candidate_pairs.remove(pair_2) #remove pair_2 from list

  final_rdd = spark.sparkContext.parallelize([]) #empty RDD

  for chunk in chunks:
    rdd_chunk = spark.sparkContext.parallelize(chunk)
    flatted_couples_chunk = rdd_chunk.map(lambda x: [(pair,1) for pair in candidate_pairs if set(pair).\
                                              issubset(set(x))]).flatMap(lambda x: x).cache()
    final_rdd = final_rdd.union(flatted_couples_chunk)


  reduced_elements = final_rdd.reduceByKey(lambda a, b: a + b)
  freq_pairs = reduced_elements.filter(lambda x : x[1] >= threshold).cache() 
  return freq_pairs


#SON(basket_file, threshold, 2).collect()

## ALGORITHMS BEHAVIOUR WITH DIFFERENT BASKET FILE SIZE AND DIFFERENT THRESHOLD

In [ ]:
times_Apriori=[]
times_PCY=[]
times_Multistage=[]
times_Multihash=[]
times_SON=[]

for size in [63656,127300,190850,254626]: #approx 1/4, 1/2, 3/4 and 4/4 of the basket file      [63656,127300,190850,254626]
  b_file= basket_file.take(size)
  b_file= spark.sparkContext.parallelize(b_file)
  _,_, time_Apriori=analysis("Apriori",b_file)
  _,_, time_PCY=analysis("PCY",b_file)
  _,_, time_Multistage=analysis("Multistage",b_file)
  _,_, time_Multihash=analysis("MULTI_HASH",b_file)
  _,_, time_SON=analysis("SON",b_file)

  times_Apriori.append([time_Apriori,"APRIORI",size])
  times_PCY.append([time_PCY,"PCY",size])
  times_Multistage.append([time_Multistage,"Multistage",size])
  times_Multihash.append([time_Multihash,"Multihash",size])
  times_SON.append([time_SON,"SON",size])


*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


In [ ]:
times_Apriori[0] #Apriori on a baskewt file of 63656 baskets

[[19.898679971694946,
  2.2798898220062256,
  1.0727570056915283,
  1.0347459316253662],
 'APRIORI',
 63656]

In [ ]:
def linearize_result (times_list_new,times_algorithm):
  i=0
  for x in times_algorithm: #cycle on the different basket sizes
    i=0
    for t in x[0]: #cycle on the different threshold
      if i==0:
        times_list_new.append([t,x[1],x[2],1000])   #[1000,3000,5000,7000]
      if i==1:
        times_list_new.append([t,x[1],x[2],3000])
      if i==2:
        times_list_new.append([t,x[1],x[2],5000])
      if i==3:
        times_list_new.append([t,x[1],x[2],7000])
      i=i+1

  return times_list_new

In [ ]:
times_list_new=[]
times_list_new=linearize_result(times_list_new,times_Apriori)
times_list_new=linearize_result(times_list_new,times_PCY)
times_list_new=linearize_result(times_list_new,times_Multistage)
times_list_new=linearize_result(times_list_new,times_Multihash)
times_list_new=linearize_result(times_list_new,times_SON)
final_df = pd.DataFrame(times_list_new, columns =['Time',"Algorithm", "Basket_File_size","Threshold"])
final_df[final_df["Algorithm"]=="PCY"	]

,Time,Algorithm,Basket_File_size,Threshold
16,3.617395,PCY,63656,1000
17,2.784158,PCY,63656,3000
18,2.522132,PCY,63656,5000
19,2.674573,PCY,63656,7000
20,8.860326,PCY,127300,1000
21,4.505386,PCY,127300,3000
22,4.328248,PCY,127300,5000
23,4.066977,PCY,127300,7000
24,16.977893,PCY,190850,1000
25,6.699015,PCY,190850,3000


## HOW THE TIME INCREAS WITH THE BASKET FILE SIZE

In [ ]:
import altair as alt
charts = []
for t in [1000,3000,5000,7000]:
  charts.append(alt.Chart(final_df[final_df["Threshold"]==t]).mark_line().encode(
      x='Basket_File_size:Q',
      y='Time:Q',
      color='Algorithm:N'
  ).properties(
      height=300,
      width=500,
      title="Threshold fixed to "+str(t)
  ))

alt.vconcat(*charts)

alt.VConcatChart(...)

## HOW THE TIME INCREASE WITH THE THRESHOLD

In [ ]:
charts2 = []
for size in [63656,127300,190850,254626]:
  charts2.append(alt.Chart(final_df[final_df["Basket_File_size"]==size]).mark_line().encode(
      x='Threshold:Q',
      y='Time:Q',
      color='Algorithm:N'
  ).properties(
      height=300,
      width=500,
      title="Basket file size fixed to "+str(size)
  ))

alt.vconcat(*charts2)

alt.VConcatChart(...)

#BRIEF ANALYS ON ASSOCIATION RULES

---



In [ ]:
def retrive_freq(singleton):
  for sing in freq_singleton_list:
    if sing[0]==singleton:
      return sing[1]

In [ ]:
freq_singleton_list[0]

('ukraineunderattaсk', 27)

In [ ]:
retrive_freq('ukraineunderattaсk')

27

In [ ]:
analisys=[]
for elem in result_PCY:
  an_row=[]
  
  support=elem[1]/500   #elem[1]=support of the pair elem[0], 500=# of baskets analyzed while perofrming PCY
  confidence_0=elem[1]/retrive_freq(elem[0][0]) #confidence of the rule A-->B
  confidence_1=elem[1]/retrive_freq(elem[0][1]) #confidence of the rule B-->A
  lift=support/( (retrive_freq(elem[0][0])/500) * (retrive_freq(elem[0][1])/500) )
  interest_0=confidence_0-(retrive_freq(elem[0][1])/500)
  interest_1=confidence_1-(retrive_freq(elem[0][0])/500)

  rule_0=elem[0][0]+"-->"+elem[0][1]
  rule_1=elem[0][1]+"-->"+elem[0][0]

  an_row.append(rule_0)
  an_row.append(support)
  an_row.append(confidence_0)
  an_row.append(lift)
  an_row.append(interest_0)
  analisys.append(an_row)

  an_row=[]
  an_row.append(rule_1)
  an_row.append(support)
  an_row.append(confidence_1)
  an_row.append(lift)
  an_row.append(interest_1)
  analisys.append(an_row)

In [ ]:
df = pd.DataFrame(analisys, columns = ['Rule','Support','Confidence','Lift','Interest'])
df=df.round(3)

In [ ]:
df

,Rule,Support,Confidence,Lift,Interest
0,protectuаsky-->stoprussia,0.040,1.000,15.625,0.936
1,stoprussia-->protectuаsky,0.040,0.625,15.625,0.585
2,russianinvasion-->standwithukraine,0.040,1.000,9.804,0.898
3,standwithukraine-->russianinvasion,0.040,0.392,9.804,0.352
4,russianinvasion-->ukrainewillwin,0.040,1.000,22.727,0.956
...,...,...,...,...,...
149,energy-->mining,0.020,1.000,50.000,0.980
150,mining-->coal,0.020,1.000,50.000,0.980
151,coal-->mining,0.020,1.000,50.000,0.980
152,ukrainerussiawar-->ukraine,0.036,0.545,1.274,0.117


A lift of more than 1 suggests that the presence of "protectuаsky" increases the probability that "stoprussia" will also occur in the transaction.